In [34]:
import os
import numpy as np
import pandas as pd
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

import shutil
from pathlib import Path

In [35]:
input_path = "/home/maderya/skill_test_m2m/dataset/"
output_path = "/home/maderya/skill_test_m2m/split_dataset2/"

In [36]:
dataset = pd.read_csv(input_path +"labels.csv")
dataset.head()

,image,front_right,front_left,rear_right,rear_left,hood
0,images/0.jpg,0,0,1,0,0
1,images/1.jpg,1,1,0,0,0
2,images/2.jpg,1,1,0,1,0
3,images/3.jpg,0,1,0,1,0
4,images/4.jpg,0,1,0,0,1


In [37]:
X = dataset["image"].values
y = dataset.iloc[:,1:].values.astype(int)
X

array(['images/0.jpg', 'images/1.jpg', 'images/2.jpg', 'images/3.jpg',
       'images/4.jpg', 'images/5.jpg', 'images/6.jpg', 'images/7.jpg',
       'images/8.jpg', 'images/9.jpg', 'images/10.jpg', 'images/11.jpg',
       'images/12.jpg', 'images/13.jpg', 'images/14.jpg', 'images/15.jpg',
       'images/16.jpg', 'images/17.jpg', 'images/18.jpg', 'images/19.jpg',
       'images/20.jpg', 'images/21.jpg', 'images/22.jpg', 'images/23.jpg',
       'images/24.jpg', 'images/25.jpg', 'images/26.jpg', 'images/27.jpg',
       'images/28.jpg', 'images/29.jpg', 'images/30.jpg', 'images/31.jpg',
       'images/32.jpg', 'images/33.jpg', 'images/34.jpg', 'images/35.jpg',
       'images/36.jpg', 'images/37.jpg', 'images/38.jpg', 'images/39.jpg',
       'images/40.jpg', 'images/41.jpg', 'images/42.jpg', 'images/43.jpg',
       'images/44.jpg', 'images/45.jpg', 'images/46.jpg', 'images/47.jpg',
       'images/48.jpg', 'images/49.jpg', 'images/50.jpg', 'images/51.jpg',
       'images/52.jpg', 'images/53.

In [38]:
# Split into train and temp
mss = MultilabelStratifiedShuffleSplit(n_splits=1,test_size=0.3, random_state=42)
train_idx, temp_idx = next(mss.split(X,y))
X_train, y_train = X[train_idx], y[train_idx]
X_temp,  y_temp = X[temp_idx],  y[temp_idx]


#split temp into test and val
mss2 = MultilabelStratifiedShuffleSplit(
    n_splits=1, test_size=0.5, random_state=42)

test_idx, val_idx = next(mss2.split(X_temp, y_temp))
X_test, y_test = X_temp[test_idx], y_temp[test_idx]
X_val,  y_val = X_temp[val_idx],  y_temp[val_idx]


In [39]:
def write_split(paths, labels, filename):
    # keep original label names
    split_df = pd.DataFrame(labels, columns=dataset.columns[1:])
    split_df.insert(0, "image_path", paths)
    split_df.to_csv(f"{output_path}/{filename}", index=False)


# Create folder if needed
os.makedirs(output_path, exist_ok=True)

write_split(X_train, y_train, "train.csv")
write_split(X_val,   y_val,   "val.csv")
write_split(X_test,  y_test,  "test.csv")

print("Splits saved:")
print(f"train: {len(X_train)} samples")
print(f"val  : {len(X_val)} samples")
print(f"test : {len(X_test)} samples")

Splits saved:
train: 255 samples
val  : 55 samples
test : 55 samples


In [40]:
import pandas as pd
import shutil
from pathlib import Path

# original folder with ALL images
DATA_ROOT = Path("/home/maderya/skill_test_m2m/dataset/")
# will contain train/val/test sub‑folders
SPLIT_ROOT = Path("split_dataset2/")
SPLIT_ROOT.mkdir(parents=True, exist_ok=True)

for split in ["train", "val", "test"]:
    (SPLIT_ROOT / split).mkdir(parents=True, exist_ok=True)


def copy_split(csv_path: str, split_name: str):
    df = pd.read_csv(csv_path)
    for img_path in df["image_path"]:
        src = DATA_ROOT / img_path          # relative path stored in CSV
        dst = SPLIT_ROOT / split_name / img_path
        # keep sub‑folders if any
        dst.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(src, dst)               # use copy2 to preserve metadata


copy_split("split_dataset2/train.csv", "train")
copy_split("split_dataset2/val.csv",   "val")
copy_split("split_dataset2/test.csv",  "test")

print("Images copied into train/val/test folders.")

Images copied into train/val/test folders.
